In [3]:
import pandas as pd 
import numpy as np 
import pickle as pkl

# Create gene id dictionary   


In [4]:
data_dir = '../data/raw/'

In [5]:
genemap = pd.read_csv(data_dir + '/geneid_map.txt', sep='\t')
#genemap = genemap[lambda x: x.Status == 'Approved' ]
genemap.head()

,HGNC ID,Approved symbol,Approved name,Status,Alias symbols,RefSeq IDs,UniProt ID(supplied by UniProt),Ensembl ID(supplied by Ensembl)
0,HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NM_130786,P04217,ENSG00000121410
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,Approved,FLJ23569,NR_015380,NaN,ENSG00000268895
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,"ACF, ASP, ACF64, ACF65, APOBEC1CF",NM_014576,Q9NQ94,ENSG00000148584
3,HGNC:7,A2M,alpha-2-macroglobulin,Approved,"FWP007, S863-7, CPAMD5",NM_000014,P01023,ENSG00000175899
4,HGNC:27057,A2M-AS1,A2M antisense RNA 1,Approved,NaN,NR_026971,NaN,ENSG00000245105


In [8]:
'TARP' in genemap['Approved symbol'].values

False

In [6]:
all2ensembl = {**{HGNC.upper() : [ensembl.upper()] for HGNC, ensembl in genemap[['HGNC ID', 
                                                    'Ensembl ID(supplied by Ensembl)']].values if type(ensembl) is not float},
           **{ensembl.upper() : [ensembl.upper()] for ensembl in genemap['Ensembl ID(supplied by Ensembl)'].values if type(ensembl) is not float},
           **{symb.upper() : [ensembl.upper()] for symb, ensembl in genemap[['Approved symbol', 'Ensembl ID(supplied by Ensembl)']].values if type(ensembl) is not float}}

for alias, ensembl in genemap[['Alias symbols', 'Ensembl ID(supplied by Ensembl)']].values: 
    if (type(alias) is not float) & (type(ensembl) is not float): 
        for alias2 in alias.split(','): 
            if alias2.upper().strip() in all2ensembl: 
                all2ensembl[alias2.upper().strip()] += [ensembl.upper()]
            else: 
                all2ensembl[alias2.upper().strip()] = [ensembl.upper()]

print('number of gene id mappings:', len(all2ensembl))
print('number of non-unique mappings:', len([x for x in all2ensembl.values() if len(x) > 1]))

with open('../data/processed/all2ensembl.pkl', 'wb') as f: 
    pkl.dump(all2ensembl, f)

number of gene id mappings: 156231
number of non-unique mappings: 1609


In [9]:
all2symb = {**{HGNC.upper() : [symb.upper()] for HGNC, symb in genemap[['HGNC ID', 
                                                    'Approved symbol']].values},

           **{symb.upper() : [symb.upper()] for symb in genemap['Approved symbol'].values},

           **{ensembl.upper() : [symb.upper()] for symb, ensembl in genemap[['Approved symbol', 'Ensembl ID(supplied by Ensembl)']].values if type(ensembl) is not float}}

for alias, symb in genemap[['Alias symbols', 'Approved symbol']].values: 
    if (type(alias) is not float) & (type(symb) is not float): 
        for alias2 in alias.split(','): 
            if alias2.upper().strip() in all2symb: 
                all2symb[alias2.upper().strip()] += [symb.upper()]
            else: 
                all2symb[alias2.upper().strip()] = [symb.upper()]

print('number of gene id mappings:', len(all2symb))
print('number of non-unique mappings:', len([x for x in all2symb.values() if len(x) > 1]))

with open('../data/processed/all2genesymbol.pkl', 'wb') as f: 
    pkl.dump(all2symb, f)

number of gene id mappings: 164349
number of non-unique mappings: 1640
